In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
folder = "musical_instruments"

In [3]:
reviews = pd.read_pickle(f"../../data/{folder}/reviews_5.p")
onehot = pd.read_pickle(f"../../data/{folder}/onehot_5.p")
sbert = pickle.load(open(f"../../data/{folder}/sbert_5.p", "rb"))

In [4]:
uuids, uiids = reviews["user_id"].unique(), reviews["asin"].unique()

In [5]:
user_map = dict(zip(uuids, list(range(1, len(uuids) + 1))))
item_map = dict(zip(uiids, list(range(1, len(uiids) + 1))))

In [6]:
pickle.dump(user_map, open(f"../../data/{folder}/usermap_5.p", "wb"))
pickle.dump(item_map, open(f"../../data/{folder}/itemmap_5.p", "wb"))

In [7]:
r_sorted = reviews.sort_values("timestamp")
r_sorted["iid"] = r_sorted["asin"].map(item_map)
r_sorted["uid"] = r_sorted["user_id"].map(user_map)

In [8]:
r_sorted["date"] = pd.to_datetime(r_sorted["timestamp"], unit="s")
r_sorted["year"], r_sorted["month"], r_sorted["day"], r_sorted["dayofweek"], r_sorted["dayofyear"], r_sorted["week"] = zip(*r_sorted["date"].map(lambda x: [x.year, x.month, x.day, x.dayofweek, x.dayofyear, x.week]))

norm_cols = ["year", "month", "day", "dayofweek", "dayofyear", "week"]
r_sorted[norm_cols] = r_sorted[norm_cols].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [9]:
ctx = {}
for t in r_sorted.itertuples():
    ctx[(t.uid, t.iid)] = [t.year, t.month, t.day, t.dayofweek, t.dayofyear, t.week]

In [10]:
onehot["iid"] = onehot["asin"].map(item_map)
onehot_np = onehot.sort_values("iid").drop(["asin", "iid"], axis=1).to_numpy()

In [11]:
sbert_t = [(k, v) for k, v in sbert.items()]
sbert_df = pd.DataFrame(sbert_t, columns=["asin", "embedding"])
sbert_df["iid"] = sbert_df["asin"].map(item_map)
sbert_np = np.stack(sbert_df.sort_values("iid")["embedding"].to_numpy())

In [12]:
len(reviews)

231312

In [14]:
reviews

,user_id,asin,timestamp
0,A1043FF66MCAW6,8288853439,1407801600
1,A2C9H5QRLXXICT,8288853439,1402531200
2,ABVQ1EZPPHZ5F,8288853439,1394928000
3,A3FMA2AFK85NMX,8288853439,1393286400
4,A2KO38Z0GOBHUD,8288853439,1392249600
...,...,...,...
1128262,A1SB9ORLJQZLEQ,B01HDZIBH8,1538265600
1128263,AQQAISCHOH5GQ,B01HFNMG10,1537747200
1128264,A20D3MES4T2LTE,B01HFNMG10,1535414400
1128265,A2ZA4Q8VJKQOQW,B01HFPO0L2,1533945600


In [13]:
with open(f"../../data/{folder}/profiles_5.txt", "w") as dst:
    for t in r_sorted.itertuples():
        uid = user_map[t.user_id]
        iid = item_map[t.asin]
        dst.write(f"{uid} {iid} {t.timestamp}\n")

In [14]:
with open(f"../../data/{folder}/ctx_5.dat", 'wb') as f:
    pickle.dump(ctx, f)

In [15]:
with open(f"../../data/{folder}/onehot_5.dat", 'wb') as f:
    pickle.dump(onehot_np, f)

In [16]:
with open(f"../../data/{folder}/sbert_5.dat", 'wb') as f:
    pickle.dump(sbert_np, f)